<a href="https://colab.research.google.com/github/Nadjei74/Collab/blob/main/RAG_PythonOCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain_community
!pip install openai
!pip install chromadb
!pip install tiktoken
!pip install python_dotenv
!pip install pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.8/384.8 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.3/361.3 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain.llms import OpenAI
import os

In [ ]:
file_path = "/content/trainset_easy.txt"
loader = TextLoader(file_path)
docs = loader.load()

In [ ]:
docs[0]

Document(metadata={'source': '/content/trainset_easy.txt'}, page_content='#Simple Shapes\ndef few_shot_examples():\n    few_shot_example = """\n### [User message]\nMake a Box with side dimension 12mm by 10mm by 7mm\n\n### [Answer]\nimport math\nfrom OCC.Display.SimpleGui import init_display\nfrom OCC.Core.gp import gp_Ax2, gp_Trsf\nfrom OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox\n\ndisplay, start_display, add_menu, add_function_to_menu = init_display()\n\nbox = BRepPrimAPI_MakeBox(12, 10, 7).Shape()\n\ndisplay.DisplayShape(box)\n\nstart_display()\n\n###########################################################################\n### [User message]\nCreate a Ball with radius of 75mm\n\n### [Answer]\nfrom OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere\nfrom OCC.Core.gp import gp_Pnt\nfrom OCC.Display.SimpleGui import init_display\n\ndisplay, start_display, add_menu, add_function_to_menu = init_display()\n\ndef create_sphere():\n    radius = 75.0\n    sphere_center = gp_Pnt(0, 0, 0)\

In [ ]:
# We use this to split the text at the ### so as to get complete individual codes/chunk
import re
from langchain.schema import Document

class CustomDelimiterTextSplitter:
    def __init__(self, min_hash_count=10):
        self.min_hash_count = min_hash_count
        self.delimiter_pattern = re.compile(r'#' * self.min_hash_count + r'+')

    def split_text(self, text):
        # Split text based on the custom delimiter pattern
        chunks = re.split(self.delimiter_pattern, text)
        return [chunk.strip() for chunk in chunks]
# Initialize the custom text splitter
custom_text_splitter = CustomDelimiterTextSplitter(min_hash_count=5)

split_docs = []
for doc in docs:
  text = doc.page_content
# Split the text
  split_texts = custom_text_splitter.split_text(text)
  split_docs.extend([Document(page_content=chunk) for chunk in split_texts])  # Create Document objects

# Print the split texts to verify
  for i, chunk in enumerate(split_texts):
    print(f"Chunk {i+1}:\n{chunk}\n")

split_texts[1]

Chunk 1:
#Simple Shapes
def few_shot_examples():
    few_shot_example = """
### [User message]
Make a Box with side dimension 12mm by 10mm by 7mm

### [Answer]
import math
from OCC.Display.SimpleGui import init_display
from OCC.Core.gp import gp_Ax2, gp_Trsf
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox

display, start_display, add_menu, add_function_to_menu = init_display()

box = BRepPrimAPI_MakeBox(12, 10, 7).Shape()

display.DisplayShape(box)

start_display()

Chunk 2:
### [User message]
Create a Ball with radius of 75mm

### [Answer]
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Core.gp import gp_Pnt
from OCC.Display.SimpleGui import init_display

display, start_display, add_menu, add_function_to_menu = init_display()

def create_sphere():
    radius = 75.0
    sphere_center = gp_Pnt(0, 0, 0)
    sphere = BRepPrimAPI_MakeSphere(sphere_center, radius).Shape()
    display.DisplayShape(sphere, update=True)

add_menu("my_menu")
add_function_to_menu("my_menu",

'### [User message]\nCreate a Ball with radius of 75mm\n\n### [Answer]\nfrom OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere\nfrom OCC.Core.gp import gp_Pnt\nfrom OCC.Display.SimpleGui import init_display\n\ndisplay, start_display, add_menu, add_function_to_menu = init_display()\n\ndef create_sphere():\n    radius = 75.0\n    sphere_center = gp_Pnt(0, 0, 0)\n    sphere = BRepPrimAPI_MakeSphere(sphere_center, radius).Shape()\n    display.DisplayShape(sphere, update=True)\n\nadd_menu("my_menu")\nadd_function_to_menu("my_menu", create_sphere)\n\nif __name__ == "__main__":\n    create_sphere()\n    start_display()'

In [ ]:
#splitting text
#text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
#texts = text_splitter.split_documents(docs)

In [ ]:
#texts[0]

NameError: name 'texts' is not defined

In [ ]:
from dotenv import load_dotenv
load_dotenv()
persist_directory = 'db'
api_key = os.environ.get("OPENAI_API_KEY")
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=split_docs,
                                 embedding=embeddings,
                                 persist_directory=persist_directory)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
#Persist the db to disk
vectordb.persist()
vectordb = None

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [ ]:
#load database
vectordb = Chroma(persist_directory=persist_directory,
                  embedding_function=embeddings)
retriever = vectordb.as_retriever(search_kwargs={"k": 10})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


In [ ]:
#Creating the fewshot template
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI


example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="{question}\n{answer}"
    )

#creatng example set
examples =[
    {
        "question": "Make a Box with side dimension 12mm by 10mm by 7mm",
        "answer": """
import math
from OCC.Display.SimpleGui import init_display
from OCC.Core.gp import gp_Ax2, gp_Trsf
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox

display, start_display, add_menu, add_function_to_menu = init_display()

box = BRepPrimAPI_MakeBox(12, 10, 7).Shape()

display.DisplayShape(box)

start_display()

              """
    },

    {

    "question": "Create a ball with radius 75mm",
    "answer":"""
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Core.gp import gp_Pnt
from OCC.Display.SimpleGui import init_display

display, start_display, add_menu, add_function_to_menu = init_display()

def create_sphere():
    radius = 75.0
    sphere_center = gp_Pnt(0, 0, 0)
    sphere = BRepPrimAPI_MakeSphere(sphere_center, radius).Shape()
    display.DisplayShape(sphere, update=True)

add_menu("my_menu")
add_function_to_menu("my_menu", create_sphere)

if __name__ == "__main__":
    create_sphere()
    start_display()

    """
    },

    {
        "question": "Create a Cone with base radius 10 mm and height 10 mm",
        "answer":"""
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeCone
from OCC.Display.SimpleGui import init_display

def create_cone(base_radius, height):
    apex_radius = 0.0
    cone = BRepPrimAPI_MakeCone(base_radius, apex_radius, height).Shape()
    return cone

def visualize_cone(cone_shape):
    display, start_display, add_menu, add_function_to_menu = init_display()
    display.DisplayShape(cone_shape, update=True)
    display.FitAll()
    start_display()

def main():
    base_radius = 10.0
    height = 10.0
    cone_shape = create_cone(base_radius, height)
    visualize_cone(cone_shape)

if __name__ == "__main__":
    main()

        """
    },
    {
        "question": "Create a Cylinder with base radius 10 mm and height 20 mm",
        "answer":"""
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeCylinder
from OCC.Display.SimpleGui import init_display

def create_and_display_cylinder(radius, height):
    cylinder = BRepPrimAPI_MakeCylinder(radius, height).Shape()
    display, start_display, add_menu, add_function_to_menu = init_display()
    display.DisplayShape(cylinder, update=True)
    display.FitAll()
    start_display()

if __name__ == "__main__":
    base_radius = 10.0
    height = 20.0
    create_and_display_cylinder(base_radius, height)


        """
    },

]

prompt =FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="""Generate a PythonOCC script for the query using the context as a reference:\n{context}.
    I only want the code as output.!DO NOT ADD ANY OTHER MESSAGE OF YOUR OWN. ANY TICKS TO THE OUTPUT""",
    suffix="Query:{query}",
    input_variables=["query","context"],
)



llm = ChatOpenAI(model_name = "gpt-4o",temperature=0)
"""qa_chain = RetrievalQA.from_llm(llm=llm,
                               retriever=retriever,
                               prompt = prompt,
                                return_source_documents=True)


"""
"""query = "Make a Box with side dimension 12mm by 10mm by 7mm"
#result = qa_chain({"query": query})
#We use the query to retrieve info from the vectorstore
retrieved_docs = retriever.get_relevant_documents(query)
context = "\n".join([doc.page_content for doc in retrieved_docs])

result = qa_chain.run({"Query":query, "context":context})
#print(result["result"]
print(result)"""



/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


'query = "Make a Box with side dimension 12mm by 10mm by 7mm"\n#result = qa_chain({"query": query})\n#We use the query to retrieve info from the vectorstore\nretrieved_docs = retriever.get_relevant_documents(query)\ncontext = "\n".join([doc.page_content for doc in retrieved_docs])\n\nresult = qa_chain.run({"Query":query, "context":context})\n#print(result["result"]\nprint(result)'

In [ ]:
# Helper function for printing docs
def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [ ]:
retrieved_doc=retriever.invoke(" Design a football (spherical in shape) of radius 20 cm ")

#print(len(retrieved_doc))
pretty_print_docs(retrieved_doc)

Document 1:

### [User message]
Create a Ball with radius of 75mm

### [Answer]
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere
from OCC.Core.gp import gp_Pnt
from OCC.Display.SimpleGui import init_display

display, start_display, add_menu, add_function_to_menu = init_display()

def create_sphere():
    radius = 75.0
    sphere_center = gp_Pnt(0, 0, 0)
    sphere = BRepPrimAPI_MakeSphere(sphere_center, radius).Shape()
    display.DisplayShape(sphere, update=True)

add_menu("my_menu")
add_function_to_menu("my_menu", create_sphere)

if __name__ == "__main__":
    create_sphere()
    start_display()
----------------------------------------------------------------------------------------------------
Document 2:

### [User message]
A bottle cap with base radius of 20mm and height of 10mm.

### [Answer]
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeCylinder
from OCC.Display.SimpleGui import init_display

def create_bottle_cap(base_radius, height):
    cylinder = BRepPrimAPI_MakeCy

In [ ]:
!pip install cohere

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.1 MB/s eta 0:00:00


In [ ]:

#Cohere API for reranking

#COHERE_API_KEY = os.environ.get("COHERE_API_KEY")
co_key ="cVOZRwnhTwq2R2HwVkLBHy8IJaF8k3zChgMAymop"
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

compressor = CohereRerank(cohere_api_key= co_key, top_n=10)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)



/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `CohereRerank` was deprecated in LangChain 0.0.30 and will be removed in 0.3.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereRerank`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [ ]:
compressed_docs = compression_retriever.invoke("  Design a football (spherical in shape) of radius 20 cm  ")
compressed_docs[0].page_content
pretty_print_docs(compressed_docs)


Document 1:

### [User message]
Design A flange with outer diameter 20mm and inner diameter 10.

### [Answer]
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeCylinder
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Cut
from OCC.Display.SimpleGui import init_display

display, start_display, add_menu, add_function_to_menu = init_display()

outer_diameter = 20.0
inner_diameter = 10.0
height = 10.0

outer_radius = outer_diameter / 2.0
outer_cylinder = BRepPrimAPI_MakeCylinder(outer_radius, height).Shape()

inner_radius = inner_diameter / 2.0
inner_cylinder = BRepPrimAPI_MakeCylinder(inner_radius, height).Shape()

flange = BRepAlgoAPI_Cut(outer_cylinder, inner_cylinder).Shape()

display.DisplayShape(flange, update=True)
start_display()
----------------------------------------------------------------------------------------------------
Document 2:

### [User message]
Design A cylinder with radius 20 mm and height 5 mm and has a square hole at the center

### [Answer]
from OCC.Core.BRepPrimAPI 

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

#Cohere API for reranking
COHERE_API_KEY = os.environ.get("COHERE_API_KEY")

#for retrieving contxt from vector base
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
#retrieved_docs = retriever.get_relevant_documents(query)
#context = "\n".join([doc.page_content for doc in retrieved_docs])

"""def print_context(inputs: dict) -> dict:
    print("Context used in RAG chain:\n", inputs['context'])
    return inputs"""
custom_prompt = prompt #We use Few_Shot_prompt
rag_chain = (
    {"context":retriever | format_docs, "query":RunnablePassthrough()}
    | custom_prompt
    | llm
    | StrOutputParser()
)



In [ ]:
query = " Design A screw with a radius of 2mm and length of 10mm. [4o F]"
response=rag_chain.invoke(query)
cleaned_code = response.replace("```python", "").replace("```", "").strip()
cleaned_code


'from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeCylinder, BRepPrimAPI_MakeCone\nfrom OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Cut\nfrom OCC.Core.gp import gp_Pnt, gp_Dir, gp_Ax2\nfrom OCC.Display.SimpleGui import init_display\n\ndef create_screw(radius, length):\n    # Create the main cylindrical body of the screw\n    cylinder = BRepPrimAPI_MakeCylinder(radius, length).Shape()\n    \n    # Create the conical tip of the screw\n    cone_height = radius * 2  # Example height for the conical tip\n    cone = BRepPrimAPI_MakeCone(gp_Ax2(gp_Pnt(0, 0, length), gp_Dir(0, 0, 1)), radius, 0, cone_height).Shape()\n    \n    # Combine the cylinder and cone to form the screw\n    screw = BRepAlgoAPI_Cut(cylinder, cone).Shape()\n    \n    return screw\n\ndef main():\n    radius = 2.0\n    length = 10.0\n    screw = create_screw(radius, length)\n    \n    display, start_display, add_menu, add_function_to_menu = init_display()\n    display.DisplayShape(screw, update=True)\n    start_display()\n\nif _

In [ ]:

template ="""
  You are a helpful AI assistant. Your job is to generate PythonOCC scripts that will generate a CAD shape
  based on the user's input. You have been given access to some examples PythonOCC scripts that will help you
  with this. When given a question, you can go through the {context} and look for the example that will help you
  generate the scripts needed to answer
  Question: {question}



"""